<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2019</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 200%;">Version 1.2</h1>
<h1>Issued Tuesday 10/01/2019; Due Thursday 10/10/2019</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be accepted, but your lowest problem-set grade will be dropped. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Do not make any edits in this cell!
class Student:
    first = "Adam"
    last = "Alston"
    onyen = "aalston9"
    pid = "730015776"

In the following questions you will need to write Python3 programs to perform queries
    on and modify the <a href="http://csbio.unc.edu/mcmillan/Media/NFL.db" download>NFL.db</a>
    database used in Problem Set #2. You sholud only need to import the standard "sqlite3"
    and the "time" Python packages.

---
**Problem 1:** The file <a href="http://csbio.unc.edu/mcmillan/Media/NFLgames2019.csv" download>NFLgames2019.csv</a>
        contains the schedule for the 2019 NFL season. It includes the scores of all games played
        before October 1, and 0-0 for games yet to be played. Write a Python3 program that loads
        these new records into the Game table, but places <i>nulls</i> for the scores of unplayed
        games.

In [1]:
import sqlite3
db = sqlite3.connect("NFL.db")
cursor = db.cursor()

filename = 'NFLgames2019.csv'
with open(filename) as fp:
    data=fp.read().split('\n')
header=data.pop(0).split(',')
for line in data:
    row = line.split(',')

    season=row[0]
    week=row[1]
    date=row[2]

    vteam=row[3].split(" ")
    vteam=vteam[len(vteam) - 1]
    cursor.execute("""select T.tid from Team T where T.mascot like (?)""", (vteam,))
    vtid=cursor.fetchone()[0]

    vscore=int(row[4])

    hteam=row[5].split(" ")
    hteam=hteam[len(hteam) - 1]
    cursor.execute("""select T.tid from Team T where T.mascot like (?)""", (hteam,))
    htid=cursor.fetchone()[0]

    hscore=int(row[6])
    notes=row[7]

    if int(row[1])<5:
        cursor.execute("""INSERT OR REPLACE INTO Game(season,week,date,vtid,vscore,htid,hscore,notes) VALUES(?,?,?,?,?,?,?,?)""",
                  (season,week,date,vtid,vscore,htid,hscore,notes,))
    if int(row[1])>=5:
        cursor.execute("""INSERT OR REPLACE INTO Game(season,week,date,vtid,htid,notes) VALUES(?,?,?,?,?,?)""",
                  (season,week,date,vtid,htid,notes,))
                                               
db.commit()
db.close()

---
**Problem 2:** Using the results from Problem #1, write a Python3 program to compute the 2019
        strength-of-schedule for each team as follows. First, find the number of wins of each team
        in 2018. Then accumulate the number of the 2018 wins of every team's opponent in the 2019
        season. Produce a sorted strength-of-schedule list with a row for each team and their number
        of wins in decreasing order. If a team is played more than once it's win-count should be
        added however many times it is played.

In [1]:
import sqlite3
import operator

db = sqlite3.connect("NFL.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()

cursor.execute ("""SELECT T.mascot FROM Team T""")
output = cursor.fetchall()
team_records = {}
for i in range (len(output)):
    Team = output[i][0]
    team_records[Team] = 0
cursor.execute("""SELECT G.htid FROM Game G WHERE season = 2018 and G.hscore > G.vscore""")
htids = cursor.fetchall()
for i in range (len(htids)):
    cursor.execute("""SELECT T.mascot FROM Team T WHERE T.tid = (?)""", (htids[i][0],))
    hteam = cursor.fetchone()[0]
    team_records[hteam] += 1
cursor.execute("""SELECT G.vtid FROM Game G WHERE season = 2018 and G.hscore < G.vscore""")
vtids = cursor.fetchall()
for i in range (len(vtids)):
    cursor.execute("""SELECT T.mascot FROM Team T WHERE T.tid = (?)""", (vtids[i][0],))
    vteam = cursor.fetchone()[0]
    team_records[vteam] += 1
cursor.execute("""SELECT G.htid, G.vtid FROM Game G WHERE G.season = 2019""")
opponents = cursor.fetchall()

opponent_wins ={}
for i in range (len(opponents)):
    cursor.execute("""SELECT t.mascot FROM Team T WHERE T.tid = (?)""", (opponents[i][0],))
    hteam = cursor.fetchone()[0]
    cursor.execute("""SELECT t.mascot FROM Team T WHERE T.tid = (?)""", (opponents[i][1],))
    vteam = cursor.fetchone()[0]
    if vteam not in opponent_wins:
        opponent_wins[vteam] = 0  
    if hteam not in opponent_wins:
        opponent_wins[hteam] = 0
        
    opponent_wins[vteam] += team_records[hteam]
    opponent_wins[hteam] += team_records[vteam]

sorted_record = dict(sorted(opponent_wins.items(), key=operator.itemgetter(0),reverse=True))
print (sorted_record)
db.close()

---
**Problem 3:** Clay Matthews and Mel Farr are examples of father-son combinations who have both
        played in the NFL, and who attended the same college. In the space below write a SQL
        query that lists these players and any others that *might* have this or a similar
        relationship.

---
**Problem 4:** If you examine the rosters that players named "Clay Matthews" and "Mel Farr" appeared
        in you will find that the database has obvious errors due to this father-son confusion.
        Write a Python3 program to repair the errant Player and PlayedFor table entries (Hint:
        you might find it useful to consult Wikipedia).

In [1]:
import sqlite3
db = sqlite3.connect("NFL.db")
cursor = db.cursor()

cursor.execute("""UPDATE PlayedFor SET pid=31227 where pid=15177 and tid=1021""")
cursor.execute("""UPDATE Player SET dob='1956-03-15' where pid=15177""")

cursor.execute("""INSERT INTO Player VALUES(31861, 'Mel Farr', '6-2',210,'UCLA','1944-11-03')""")
cursor.execute("""UPDATE PlayedFor SET pid=31861 WHERE pid=12209 AND tid=1020""")

db.commit()
db.close()

---
**Problem 5:** The folowing query was designed to identify players with the same name who have played for
        the same team:<br>
        <pre>
        SELECT *
        FROM Player P1, PlayedFor R1, Player P2, PlayedFor R2
        WHERE R1.pid=P1.pid AND R2.pid=P2.pid
        AND R1.tid=R2.tid AND R1.pid&lt;R2.pid AND P1.name=P2.name
        GROUP BY P1.pid, P2.pid
        </pre>
        This query is slow using the default NFL.db. Write a Python3 program to execute and time
        this query and the time to save all rows that it returns into list for subsequent processing.
        You should verify that your list actually extracts data from the rows. In the cell provided
        below give the number of rows returned and the time required (no code please).

---
**Problem 6:** Next try to create and index to improve this query. In the cell provided below give
        the "CREATE INDEX" statement that you found that provides the best performance. Also include
        the number of rows returned and the time required for the query after creating the index
        you chose.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. 